In [1]:
import os
import pandas as pd
import numpy as np
import uuid
import random
from datetime import datetime, timedelta

import values, objects, helpers
import tables

In [2]:
all_tables = tables.all_tables

In [3]:
version = str(datetime.now())
os.mkdir(f'data/{version}')

for t in all_tables:
    for k, v in t.items():
        v.to_csv(f'data/{version}/{k}.csv', index=False)

In [16]:

VBAK.to_csv(f'data/{version}/VBAK.csv', index=False) 

In [3]:
vbak_len = 10_000

In [4]:
# Materials
mara_temp_list = []
MATERIALS = []
for mat in values.material_names:
    rand_id = uuid.uuid4()
    mara_temp_list.append([values.mandt, rand_id, mat['name']])
    MATERIALS.append(objects.Materials(id=rand_id, name=mat['name'], availability=mat['availability'], price=mat['price']))

mara =  pd.DataFrame(mara_temp_list, columns=mara_cols)

In [5]:
# Users
usr01_temp_list = []
USERS = []
for u_name in values.users:
    usr01_temp_list.append([values.mandt, u_name, 'A'])
    USERS.append(objects.Users(u_name, 'A'))
usr01_temp_list.append([values.mandt, 'BATCH_JOB', 'B'])
USERS.append(objects.Users('BATCH_JOB', 'B'))

usr01 = pd.DataFrame(usr01_temp_list, columns=usr01_cols)

In [6]:
# Customers
kna1_temp_list = []
COMPANIES = []
for company in values.company_names:
    rand_id = uuid.uuid4()
    rand_dt = values.times['customer_create_start'] + helpers.UPTO_YEAR()
    rand_usr = random.choice(list(values.users))
    kna1_temp_list.append([values.mandt, rand_id, company['name'], rand_dt, rand_usr])
    COMPANIES.append(objects.Customer(rand_id, rand_dt, company['credit_worthyness']))

kna1 = pd.DataFrame(kna1_temp_list, columns=Kna1_cols)

In [7]:
vbak_temp_list = []
vbap_temp_list = []
likp_temp_list = []
lips_temp_list = []
cdhdr_changes_temp_list = []
cdpos_changes_temp_list = []
vbuk_temp_list = []
mkpf_temp_list = []
mseg_temp_list = []
vbrk_temp_list = []
vbrp_temp_list = []

for i in range(vbak_len):
    vbak_obj = objects.SalesOrder(users=USERS, companies=COMPANIES, materials=MATERIALS)
    billing_dev_obj = objects.BillingDeviation(vbak=vbak_obj)
    cancel_return_dev_obj = objects.CancelOrReturn(vbak=vbak_obj)

    activities = [
        vbak_obj.generate_delivery_doc, 
        vbak_obj.release_delivery, 
        billing_dev_obj.set_billing_block, 
        vbak_obj.ship_goods, 
        billing_dev_obj.remove_billing_block, 
        vbak_obj.send_invoice, 
        cancel_return_dev_obj.cancel_order, 
        vbak_obj.receive_delivery_confirmation, 
        cancel_return_dev_obj.return_goods, 
        vbak_obj.clear_invoice,
        'end'
    ]

    step = 0
    while step < len(activities) - 1:
        # take a step
        activities[step]()
        vbak_obj.transition_matrix = vbak_obj.transition_matrix / vbak_obj.transition_matrix.sum(axis=1, keepdims=True)
                
        # step = next_step
        step = np.random.choice(len(activities), p=vbak_obj.transition_matrix[step]) # select another step

        # break if end
        if step == len(activities) - 1:
                break

    vbuk_temp_list.append(vbak_obj.vbuk)
    vbak_temp_list.append([values.mandt, vbak_obj.vbeln, vbak_obj.erdat, vbak_obj.ernam, vbak_obj.kunnr, vbak_obj.netwr, vbak_obj.vsbed, vbak_obj.faksk])
    vbap_temp_list += vbak_obj.vbap_temp_list
    likp_temp_list += vbak_obj.likp_temp_list
    lips_temp_list += vbak_obj.lips_temp_list
    cdhdr_changes_temp_list += vbak_obj.cdhdr_changes_temp_list
    cdpos_changes_temp_list += vbak_obj.cdpos_changes_temp_list
    mkpf_temp_list += vbak_obj.mkpf_temp_list
    mseg_temp_list += vbak_obj.mseg_temp_list
    vbrk_temp_list += vbak_obj.vbrk_temp_list
    vbrp_temp_list += vbak_obj.vbrp_temp_list

In [8]:
vbak = pd.DataFrame(vbak_temp_list, columns=vbak_cols)
vbap = pd.DataFrame(vbap_temp_list, columns=vbap_cols)
likp = pd.DataFrame(likp_temp_list, columns=likp_cols)
lips = pd.DataFrame(lips_temp_list, columns=lips_cols)
cdhdr = pd.DataFrame(cdhdr_changes_temp_list, columns=cdhdr_cols)
cdpos = pd.DataFrame(cdpos_changes_temp_list, columns=cdpos_cols)
vbuk = pd.DataFrame(vbuk_temp_list, columns=vbuk_cols)
mkpf = pd.DataFrame(mkpf_temp_list, columns=mkpf_cols)
mseg = pd.DataFrame(mseg_temp_list, columns=mseg_cols)
vbrk = pd.DataFrame(vbrk_temp_list, columns=vbrk_cols)
vbrp = pd.DataFrame(vbrp_temp_list, columns=vbrp_cols)

##### Save csv

In [9]:
version = str(datetime.now())
os.mkdir(f'data/{version}')
cdhdr.to_csv(f'data/{version}/cdhdr.csv', index=False) 
cdpos.to_csv(f'data/{version}/cdpos.csv', index=False) 
mara.to_csv(f'data/{version}/mara.csv', index=False) 
usr01.to_csv(f'data/{version}/usr01.csv', index=False) 
kna1.to_csv(f'data/{version}/kna1.csv', index=False) 
vbak.to_csv(f'data/{version}/vbak.csv', index=False) 
vbap.to_csv(f'data/{version}/vbap.csv', index=False) 
likp.to_csv(f'data/{version}/likp.csv', index=False) 
lips.to_csv(f'data/{version}/lips.csv', index=False) 
vbuk.to_csv(f'data/{version}/vbuk.csv', index=False) 
mkpf.to_csv(f'data/{version}/mkpf.csv', index=False) 
mseg.to_csv(f'data/{version}/mseg.csv', index=False) 
vbrk .to_csv(f'data/{version}/vbrk.csv', index=False) 
vbrp .to_csv(f'data/{version}/vbrp.csv', index=False) 

In [10]:
vbap

,MANDT,VBELN,POSNR,KWMENG,MATNR,NETWR
0,SC1,00f50b55-d5a5-4448-aa59-7f65576cead4,0,39,f6d19a84-c5ad-453d-be6b-0aed9c923e07,28080.0
1,SC1,00f50b55-d5a5-4448-aa59-7f65576cead4,1,63,d809e0f0-e25c-4fa8-bc50-661d6ee8e24b,54432.0
2,SC1,00f50b55-d5a5-4448-aa59-7f65576cead4,2,133,dfd2b269-d204-49ca-854e-3fdc9d90d6e6,63201.6
3,SC1,00f50b55-d5a5-4448-aa59-7f65576cead4,3,105,a53679c9-5b9a-40e7-a19d-2777bd3d73fa,226800.0
4,SC1,00f50b55-d5a5-4448-aa59-7f65576cead4,4,62,f2abfeb9-14b0-40af-88fb-f6cac26b55bb,74102.4
...,...,...,...,...,...,...
100087,SC1,22f82c71-e056-4ba4-bcf9-ea6d77a32170,1,133,e2755dc2-5156-42ba-9de1-fbcc962b4309,153216.0
100088,SC1,22f82c71-e056-4ba4-bcf9-ea6d77a32170,2,71,09629526-6b0a-430e-ba15-5fd878c8c786,15336.0
100089,SC1,22f82c71-e056-4ba4-bcf9-ea6d77a32170,3,75,e54227b9-71b8-4a78-8600-66a150cb264c,22680.0
100090,SC1,22f82c71-e056-4ba4-bcf9-ea6d77a32170,4,73,f2abfeb9-14b0-40af-88fb-f6cac26b55bb,87249.6
